## ADF Diagnostics In Jupyter
This notebook will run the Atmospheric Diagnostic Framework using the settings in a config.yaml file in your ADF directory. 

Note that it was developed to run on Cheyenne/Caspar JupyterHub *with the NPL (conda) kernel*

### Setup
#### Required packages

In [1]:
import os.path
import sys


#### Paths

In [2]:
# ADF Code path
# If it is in your cwd, set adf_code = local_path (initialized below)
# otherwise set adf_code appropriately

local_path = os.path.abspath('')
adf_code = local_path
#adf_code = "/glade/u/home/bundy/diag/ADF_top/ADF"

#set path to ADF lib (main code)
lib_path = os.path.join(adf_code,"lib")

#set path to ADF plotting scripts 
plotting_scripts_path = os.path.join(adf_code,"scripts","plotting")

#Add paths to python path:
sys.path.append(lib_path)
sys.path.append(plotting_scripts_path)

print(f"current working directory = {local_path}")
print(f"ADF path                  = {adf_code}")
print(f"ADF source code           = {lib_path}")
print(f"ADF plotting scripts      = {plotting_scripts_path}")


current working directory = /glade/work/nusbaume/SE_projects/model_diagnostics/bitterbark_ADF
ADF path                  = /glade/work/nusbaume/SE_projects/model_diagnostics/bitterbark_ADF
ADF source code           = /glade/work/nusbaume/SE_projects/model_diagnostics/bitterbark_ADF/lib
ADF plotting scripts      = /glade/work/nusbaume/SE_projects/model_diagnostics/bitterbark_ADF/scripts/plotting


#### Paths to data that is input or produced by the ADF are set in the config.yaml file. 
To modify from the defaults, _edit that file_.

If there are errors here, it is likely due to path errors above



In [3]:
#set path to config YAML file:
config_file=os.path.join(adf_code,"config_cam_baseline_example.yaml")

print(f"Will read settings from ",config_file)

Will read settings from  /glade/work/nusbaume/SE_projects/model_diagnostics/bitterbark_ADF/config_cam_baseline_example.yaml


In [4]:
#import ADF diagnostics object
from adf_diag import AdfDiag

# If this fails, check your paths output in the cells above,
# and that you are running the NPL (conda) Kernel
# You can see all the paths being examined by un-commenting the following:
#sys.path

In [5]:
#
# Initialize ADF object from the user-set config file
#
# Note that you will need to set 'user' in your config file. 
# The file makes some assumptions about output/working data directories 
# that can be checked by searching for where the variable 'user' is used.
#
# Any changes that are made to the config file will require re-running this cell
# 
adf = AdfDiag(config_file)
print(f"Reading settings from ",config_file)

check_user = adf.read_config_var("user")
err_msg = 'ERROR: You need to set user in the config.yaml to your user name'
assert check_user != 'USER-NAME-NOT-SET', f'{err_msg}'
print(f'user name set to: {check_user}')


Reading settings from  /glade/work/nusbaume/SE_projects/model_diagnostics/bitterbark_ADF/config_cam_baseline_example.yaml
user name set to: nusbaume


In [6]:
# Pull (and print) some info from the ADF object:
baseline_dict = adf.read_config_var("diag_cam_climo")
baseline_dict

{'calc_cam_climo': True,
 'cam_overwrite_climo': False,
 'cam_case_name': 'b.e20.BHIST.f09_g17.20thC.297_05',
 'cam_hist_loc': '/glade/p/cesm/ADF/${diag_cam_climo.cam_case_name}',
 'cam_climo_loc': '/glade/scratch/${user}/ADF/${diag_cam_climo.cam_case_name}/climo',
 'start_year': 1990,
 'end_year': 1999,
 'cam_ts_done': False,
 'cam_ts_save': True,
 'cam_overwrite_ts': False,
 'cam_ts_loc': '/glade/scratch/${user}/ADF/${diag_cam_climo.cam_case_name}/ts'}

In [11]:
# More useful to get information with the variable settings (eg cam_case_name) replaced
# with their actual values:

#Grab a processed variable under "diag_basic_info":
print("ADF plots will be written to",adf.get_basic_info('cam_diag_plot_loc', required=True))

#Grab a processed variable under "diag_cam_climo":
print("case hist files will be read from",adf.get_cam_info('cam_hist_loc', required=True))

#Grab a processed variable under "diag_cam_baseline_climo":
print("baseline time series files will be read from",adf.get_baseline_info('cam_ts_loc', required=True))

#Finally, if you request a non-existent variable, and don't have "required=True", it will return None:
print("The number of pineapples in the config file is",adf.get_basic_info('pineapples'))

ADF plots will be written to /glade/scratch/nusbaume/ADF/plots
case hist files will be read from ['/glade/p/cesm/ADF/b.e20.BHIST.f09_g17.20thC.297_05']
baseline time series files will be read from /glade/scratch/nusbaume/ADF/b.e20.BHIST.f09_g16.20thC.125.02/ts
The number of pineapples in the config file is None


### ADF Standard Work Flow

In [12]:
#Create model time series.
print(f"case ts files will be created in",adf.get_cam_info('cam_ts_loc', required=True))
adf.create_time_series()


   

case ts files will be created in ['/glade/scratch/nusbaume/ADF/b.e20.BHIST.f09_g17.20thC.297_05/ts']

  Generating CAM time series files...
	 Processing time series for case 'b.e20.BHIST.f09_g17.20thC.297_05' :
	 - time series for SWCF
	 - time series for LWCF
	 - time series for PRECC
	 - time series for PRECL
	 - time series for PSL
	 - time series for Q
	 - time series for U
	 - time series for T
	 - time series for RELHUM
	 - time series for TREFHT
	 - time series for TS
	 - time series for TAUX
	 - time series for TAUY
	 - time series for FSNT
	 - time series for FLNT
	 - time series for LANDFRAC
  ...CAM time series file generation has finished successfully.


In [15]:
#Create model baseline time series (if needed):
if not adf.compare_obs:
    print(f"baseline ts files will be created in",adf.get_baseline_info('cam_ts_loc', required=True))
    adf.create_time_series(baseline=True)


baseline ts files will be created in /glade/scratch/nusbaume/ADF/b.e20.BHIST.f09_g16.20thC.125.02/ts

  Generating CAM time series files...
	 Processing time series for case 'b.e20.BHIST.f09_g16.20thC.125.02' :
	 - time series for SWCF
	 - time series for LWCF
	 - time series for PRECC
	 - time series for PRECL
	 - time series for PSL
	 - time series for Q
	 - time series for U
	 - time series for T
	 - time series for RELHUM
	 - time series for TREFHT
	 - time series for TS
	 - time series for TAUX
	 - time series for TAUY
	 - time series for FSNT
	 - time series for FLNT
	 - time series for LANDFRAC
  ...CAM time series file generation has finished successfully.


In [16]:
#Create model climatology (climo) files.
print(f"case climo files will be created in",adf.get_cam_info('cam_climo_loc', required=True))
adf.create_climo()

case climo files will be created in ['/glade/scratch/nusbaume/ADF/b.e20.BHIST.f09_g17.20thC.297_05/climo']

  Calculating CAM climatologies...
	 Calculating climatologies for case 'b.e20.BHIST.f09_g17.20thC.297_05' :
	    INFO: Found climo file and clobber is False, so skipping SWCF and moving to next variable.
	    INFO: Found climo file and clobber is False, so skipping LWCF and moving to next variable.
	    INFO: Found climo file and clobber is False, so skipping PRECC and moving to next variable.
	    INFO: Found climo file and clobber is False, so skipping PRECL and moving to next variable.
	    INFO: Found climo file and clobber is False, so skipping PSL and moving to next variable.
	    INFO: Found climo file and clobber is False, so skipping Q and moving to next variable.
	    INFO: Found climo file and clobber is False, so skipping U and moving to next variable.
	    INFO: Found climo file and clobber is False, so skipping T and moving to next variable.
	    INFO: Found climo 

In [17]:
#If a user is doing a model vs obs comparison, but
#no observations were found, then stop here:
if adf.compare_obs and not adf.var_obs_dict:
        print('ADF diagnostics has completed successfully.')
        sys.exit(0)
else:
    print('config file did not ask ADF to compare obs')

config file did not ask ADF to compare obs


In [18]:
#Regrid model climatology files to match either
#observations or CAM baseline climatologies.
#This call uses the "regridding_scripts" specified
#in the config file:
adf.regrid_climo()

 


  Regridding CAM climatologies...
    /glade/scratch/nusbaume/ADF/regrid not found, making new directory
	 Regridding case 'b.e20.BHIST.f09_g17.20thC.297_05' :
	 - regridding SWCF (known targets: ['b.e20.BHIST.f09_g16.20thC.125.02'])
	 - regridding LWCF (known targets: ['b.e20.BHIST.f09_g16.20thC.125.02'])
	 - regridding PRECC (known targets: ['b.e20.BHIST.f09_g16.20thC.125.02'])
	 - regridding PRECL (known targets: ['b.e20.BHIST.f09_g16.20thC.125.02'])
	 - regridding PSL (known targets: ['b.e20.BHIST.f09_g16.20thC.125.02'])
	 - regridding Q (known targets: ['b.e20.BHIST.f09_g16.20thC.125.02'])
Please ignore the interpolation warnings that follow!


Interpolation point out of data bounds encountered
Interpolation point out of data bounds encountered


Please ignore the interpolation warnings that follow!


Interpolation point out of data bounds encountered
Interpolation point out of data bounds encountered


	 - regridding U (known targets: ['b.e20.BHIST.f09_g16.20thC.125.02'])
Please ignore the interpolation warnings that follow!


...

	 - regridding RELHUM (known targets: ['b.e20.BHIST.f09_g16.20thC.125.02'])
Please ignore the interpolation warnings that follow!


Interpolation point out of data bounds encountered
Interpolation point out of data bounds encountered


Please ignore the interpolation warnings that follow!


Interpolation point out of data bounds encountered
Interpolation point out of data bounds encountered


	 - regridding TREFHT (known targets: ['b.e20.BHIST.f09_g16.20thC.125.02'])
	 - regridding TS (known targets: ['b.e20.BHIST.f09_g16.20thC.125.02'])
	 - regridding TAUX (known targets: ['b.e20.BHIST.f09_g16.20thC.125.02'])
	 - regridding TAUY (known targets: ['b.e20.BHIST.f09_g16.20thC.125.02'])
	 - regridding FSNT (known targets: ['b.e20.BHIST.f09_g16.20thC.125.02'])
	 - regridding FLNT (known targets: ['b.e20.BHIST.f09_g16.20thC.125.02'])
	 - regridding LANDFRAC (known targets: ['b.e20.BHIST.f09_g16.20thC.125.02'])
  ...CAM climatologies have been regridded successfully.


In [19]:
#Perform analyses on the simulation(s).
#This call uses the "analysis_scripts" specified in the
#config file:
adf.perform_analyses()


  Calculating AMWG variable table...
	    /glade/scratch/nusbaume/ADF/plots/b.e20.BHIST.f09_g17.20thC.297_05_1990_1999_vs_b.e20.BHIST.f09_g16.20thC.125.02_1990_1999 not found, making new directory
	 - Variable 'SWCF' being added to table
	 - Variable 'LWCF' being added to table
	 - Variable 'PRECC' being added to table
	 - Variable 'PRECL' being added to table
	 - Variable 'PSL' being added to table
	 - Variable 'Q' being added to table
	   Variable 'Q' has a vertical dimension, which is currently not supported for the AMWG Table. Skipping...
	 - Variable 'U' being added to table
	   Variable 'U' has a vertical dimension, which is currently not supported for the AMWG Table. Skipping...
	 - Variable 'T' being added to table
	   Variable 'T' has a vertical dimension, which is currently not supported for the AMWG Table. Skipping...
	 - Variable 'RELHUM' being added to table
	   Variable 'RELHUM' has a vertical dimension, which is currently not supported for the AMWG Table. Skipping...
	 

In [20]:
#Create the plot requested in the plotting_scripts list in the config file:
plot_path = adf.get_basic_info('cam_diag_plot_loc')
print(f'plots written to {plot_path}')
adf.create_plots()

plots written to /glade/scratch/nusbaume/ADF/plots

  Generating lat/lon maps...
	 NOTE: Plot type is set to png
	 NOTE: redo_plot is set to False
	 - lat/lon maps for SWCF
	 - lat/lon maps for LWCF
	 - lat/lon maps for PRECC
	 - lat/lon maps for PRECL
	 - lat/lon maps for PSL
	 - lat/lon maps for Q
	 - lat/lon maps for U
	 - lat/lon maps for T
	 - lat/lon maps for RELHUM
	 - lat/lon maps for TREFHT
	 - lat/lon maps for TS
	 - lat/lon maps for TAUX
	 - lat/lon maps for TAUY
	 - lat/lon maps for FSNT
	 - lat/lon maps for FLNT
	 - lat/lon maps for LANDFRAC
  ...lat/lon maps have been generated successfully.

  Generating lat/lon vector maps...
	 NOTE: redo_plot is set to False
	 - lat/lon vector maps for U,V
	 ERROR: Did not find any oclim_fils. Will try to skip.
	 INFO: Data Location, dclimo_loc is /glade/scratch/nusbaume/ADF/regrid
	 INFO: The glob is: b.e20.BHIST.f09_g16.20thC.125.02_V_*.nc
	 - lat/lon vector maps for TAUX,TAUY
  ...lat/lon vector maps have been generated successfully

KeyboardInterrupt: 

In [15]:
#Create website.
if adf.create_html:
    adf.create_website()


  Generating Diagnostics webpages...
  ...Webpages have been generated successfully.


### ADF Helpful Methods and Structures 

#### Demonstration of a few methods to get information from the ADF object

In [16]:
basic_info_dict = adf.read_config_var("diag_basic_info")
print(basic_info_dict)




{'compare_obs': False, 'create_html': True, 'obs_data_loc': '/glade/work/nusbaume/SE_projects/model_diagnostics/ADF_obs', 'cam_regrid_loc': '/glade/scratch/${user}/ADF/regrid', 'cam_overwrite_regrid': False, 'cam_diag_plot_loc': '/glade/scratch/${user}/ADF/plots', 'use_defaults': True, 'plot_press_levels': [200, 850], 'central_longitude': 180, 'weight_season': True, 'num_procs': 8, 'redo_plot': False}


In [17]:
baseline_dict = adf.read_config_var("diag_cam_baseline_climo")
print(baseline_dict)

{'calc_cam_climo': True, 'cam_overwrite_climo': False, 'cam_case_name': 'b.e20.BHIST.f09_g16.20thC.125.02', 'cam_hist_loc': '/glade/p/cesm/ADF/${diag_cam_baseline_climo.cam_case_name}', 'cam_climo_loc': '/glade/scratch/${user}/ADF/${diag_cam_baseline_climo.cam_case_name}/climo', 'start_year': 1990, 'end_year': 1999, 'cam_ts_done': False, 'cam_ts_save': True, 'cam_overwrite_ts': False, 'cam_ts_loc': '/glade/scratch/${user}/ADF/${diag_cam_baseline_climo.cam_case_name}/ts'}


In [18]:
case_names = adf.get_cam_info("cam_case_name",required=True)
print(case_names)

['b.e20.BHIST.f09_g17.20thC.297_05']


In [19]:
plot_type = basic_info_dict.get('plot_type', 'png')
plot_type

'png'

In [35]:
case_climo_loc = adf.get_cam_info('cam_climo_loc', required=True)

# Variables used in a CAM v CAM (baseline comparison), not required because not used if compare_obs = True
data_name = adf.get_baseline_info('cam_case_name', required=False)
data_loc = adf.get_baseline_info("cam_climo_loc", required=False)


In [36]:
var_list = adf.diag_var_list
print(var_list)

['SWCF', 'LWCF', 'PRECC', 'PRECL', 'PSL', 'Q', 'U', 'T', 'RELHUM', 'TREFHT', 'TS', 'TAUX', 'TAUY', 'FSNT', 'FLNT', 'LANDFRAC']


 #### Demonstrate how to check for a variable in the list 

In [37]:
#Print all the variables
var_list

['SWCF',
 'LWCF',
 'PRECC',
 'PRECL',
 'PSL',
 'Q',
 'U',
 'T',
 'RELHUM',
 'TREFHT',
 'TS',
 'TAUX',
 'TAUY',
 'FSNT',
 'FLNT',
 'LANDFRAC']

In [38]:
# Check for a variable
var_name = 'PRECC'
assert var_name in var_list, f'Sorry, you need to include {var_name} to make this plot'

In [39]:
# Check for a variable that isn't in the list
var_name = 'PRECT'
assert var_name in var_list, f'Sorry, you need to include {var_name} to make this plot'

# EXPECT AN ERROR for demonstration purposes! 


AssertionError: Sorry, you need to include PRECT to make this plot